# Evaluation using BERTscore

[Source](https://huggingface.co/spaces/evaluate-metric/bertscore)

In [1]:
import pandas as pd

df = pd.read_csv("../agent/predictions.csv")
df.head()

,input,biases,scores,debias_reasoning,output,index,url
0,Horario de atención de lunes a jueves de 09:00...,UNBIASED,1.0,NaN,UNBIASED,21000,detalle?id=8391
1,"El Segundo proceso de modificación, permanecer...",UNBIASED,1.0,NaN,UNBIASED,21001,detalle?id=8389
2,Unidad de Calidad de Vida Estudiantil te invit...,UNBIASED,1.0,NaN,UNBIASED,21002,detalle?id=8385
3,Es necesario el compromiso de acompañar a un m...,UNBIASED,1.0,NaN,UNBIASED,21003,detalle?id=8385
4,Cuenta con un pago mensual.,UNBIASED,1.0,NaN,UNBIASED,21004,detalle?id=8385


In [2]:
from pathlib import Path

data_path = '../../data/augmented'
file_path = Path(data_path)

dataframes = []
for file in file_path.glob('*.xlsx'):
    df_file = pd.read_excel(file)
    dataframes.append(df_file)

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.head()

,index,titulo,url,mensaje,fecha,frases,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,potencialmente_sesgable_otro_sesgo,revisado,description_bias_removal,biased_words,unbiased_words
0,13000,Micro-curso OpenLab Salud: desafío en interope...,detalle?id=25408,Estimad@s estudiantes: \n\n\n\nEl área de form...,2019-04-03 15:05:03,Quiénes pueden participar: Estudiantes de preg...,SI,NO,Quiénes pueden participar: Los Estudiantes de ...,Quiénes pueden participar: Estudiantes de preg...,NO,SI,"Se eliminó el artículo ""Los"" para incluir a es...",Los,Estudiantes
1,13001,Micro-curso OpenLab Salud: desafío en interope...,detalle?id=25408,Estimad@s estudiantes: \n\n\n\nEl área de form...,2019-04-03 15:05:03,Estudiantes de pregrado de la Facultad de Medi...,SI,NO,Alumnos de pregrado de la Facultad de Medicina...,Estudiantes de pregrado de la Facultad de Medi...,NO,SI,"Se reemplazó ""Alumnos"" por ""Estudiantes"" para ...",Alumnos,Estudiantes
2,13002,Micro-curso OpenLab Salud: desafío en interope...,detalle?id=25408,Estimad@s estudiantes: \n\n\n\nEl área de form...,2019-04-03 15:05:03,Estudiantes emprendedores.,SI,NO,Alumnos emprendedores.,Estudiantes emprendedoras/es,NO,SI,"Se reemplazó ""Alumnos"" por ""Estudiantes empren...",Alumnos,Estudiantes emprendedoras/es
3,13003,Micro-curso OpenLab Salud: desafío en interope...,detalle?id=25408,Estimad@s estudiantes: \n\n\n\nEl área de form...,2019-04-03 15:05:03,*Todos los jueves del 25 de abril al 6 de juni...,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN
4,13004,Micro-curso OpenLab Salud: desafío en interope...,detalle?id=25408,Estimad@s estudiantes: \n\n\n\nEl área de form...,2019-04-03 15:05:03,Revisa el programa completo https://bit.ly/2U...,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN


In [3]:
combined_df['sesgo_pronombre'].describe()

count     4668
unique       7
top         NO
freq      2604
Name: sesgo_pronombre, dtype: object

In [4]:
combined_df['sesgo_pronombre'].unique()

array(['SI', nan, 'SI ', 'NO', 'SI0', 'Si', 'No', 'a'], dtype=object)

In [5]:
print(f'Original length = {len(df)}')

df = df.dropna(subset=['biases'])

print(f'Length after removing NaNs = {len(df)}')

Original length = 15959
Length after removing NaNs = 948


In [ ]:
merged_df = pd.merge(df, combined_df, on='index', how='inner')
len(merged_df)

948

In [7]:
df_corrected = merged_df[merged_df['output']!='UNBIASED']
print(f'Number of corrected rows {len(df_corrected)}')

Number of corrected rows 290


Number of rows where the LLM corrected when it wasn't needed

In [8]:
len(df_corrected[(df_corrected['sesgo_pronombre']=='NO') & (df_corrected['sesgo_otro']=='NO')])

21

In [9]:
df_corrected[(df_corrected['sesgo_pronombre']=='NO') & (df_corrected['sesgo_otro']=='NO')].index

Index([ 52,  80, 107, 159, 160, 277, 282, 286, 290, 312, 370, 376, 472, 519,
       520, 727, 728, 759, 900, 915, 940],
      dtype='int64')

A particular example where the model has corrected when it was not needed

In [12]:
print('input: ',df_corrected[(df_corrected['sesgo_pronombre']=='NO') & (df_corrected['sesgo_otro']=='NO')]['input'][52])
print('output: ',df_corrected[(df_corrected['sesgo_pronombre']=='NO') & (df_corrected['sesgo_otro']=='NO')]['output'][52])

input:  Para quienes no hayan renovado su fotografía para el proceso de Retarjetización, las direcciones y horarios de oficinas de atención JUNAEB son: Horario atención: lunes a viernes de 9:00 a 18:00 hrs.
output:  Para las personas que no hayan renovado su fotografía para el proceso de Retarjetización, las direcciones y horarios de oficinas de atención JUNAEB son: Horario atención: lunes a viernes de 9:00 a 18:00 hrs.


In [13]:
print('input: ',df_corrected[(df_corrected['sesgo_pronombre']=='NO') & (df_corrected['sesgo_otro']=='NO')]['input'][107])
print('output: ',df_corrected[(df_corrected['sesgo_pronombre']=='NO') & (df_corrected['sesgo_otro']=='NO')]['output'][107])

input:  Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.
output:  Se informa a toda la comunidad estudiantil que las personas que NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, no podrán obtener esta nueva versión desde el próximo año.


### Example

There are potentially many options for replacing a biased text. We will consider the highest similarity among all candidate options.

In [14]:
references = ['Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.']
predictions = ['Se informa a toda la comunidad estudiantil que las personas que NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, no podrán obtener esta nueva versión desde el próximo año.' for _ in references]

In [15]:
from evaluate import load
bertscore = load("bertscore")

In [16]:
results = bertscore.compute(predictions=predictions, references=references, model_type="bert-base-multilingual-cased")
print(results)

{'precision': [0.9533587098121643], 'recall': [0.9690830707550049], 'f1': [0.9611566066741943], 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.47.1)'}


Full evaluation

In [17]:
merged_df['version_sin_sesgo']

0                                                    NaN
1                                                    NaN
2      Unidad de Calidad de Vida Estudiantil les invi...
3      Es necesario el compromiso de acompañar a una/...
4                                                    NaN
                             ...                        
943                                                  NaN
944                                                  NaN
945                                                  NaN
946    El Centro de Estudiantes de Ingeniería (CEI) l...
947                                                  NaN
Name: version_sin_sesgo, Length: 948, dtype: object

In [18]:
merged_df = merged_df.dropna(subset=['version_sin_sesgo'])
print(len(merged_df))

333


In [21]:
results = bertscore.compute(predictions=list(merged_df['output']), references=list(merged_df['version_sin_sesgo']), model_type="bert-base-multilingual-cased")

In [23]:
merged_df['precision'] = results['precision']
merged_df['recall'] = results['recall']
merged_df['f1'] = results['f1']

In [24]:
len(merged_df)

333

In [25]:
def filter_best_scores(df, id_column, score_column):
    """
    Reduces the dataframe to only the rows with the best score for each ID.

    Parameters:
    - df (pd.DataFrame): The original dataframe
    - id_column (str): Name of the column containing unique IDs
    - score_column (str): Name of the column containing the scores

    Returns:
    - pd.DataFrame: A filtered dataframe with the best score per ID
    """
    # Find the maximum score for each ID_row
    best_scores = df.groupby(id_column)[score_column].transform('max')
    
    # Filter the rows where the score matches the maximum score for each ID_row
    filtered_df = df[df[score_column] == best_scores].copy(deep=True)
    
    return filtered_df

In [26]:
df_results = filter_best_scores(merged_df, 'index', 'f1')
len(df_results)

333

In [27]:
import numpy as np

for k in results.keys():
    if not k == 'hashcode':
        print(f'{k}',f'\n\tmean: {np.mean(df_results[k])}\n\tstd: {np.std(df_results[k])}\n')

precision 
	mean: 0.8309580647909606
	std: 0.16464016886510133

recall 
	mean: 0.7824891748371067
	std: 0.20287524738079862

f1 
	mean: 0.8046059384718314
	std: 0.1852749454425095



Subtracting input

In [30]:
results_input = bertscore.compute(
    predictions=list(df_results['input']),
    references=list(df_results['version_sin_sesgo']), model_type="bert-base-multilingual-cased")
df_results['precision_input'] = results_input['precision']
df_results['recall_input'] = results_input['recall']
df_results['f1_input'] = results_input['f1']

df_results['precision_diff'] = df_results['precision'] - df_results['precision_input']
df_results['recall_diff'] = df_results['recall'] - df_results['recall_input']
df_results['f1_diff'] = df_results['f1'] - df_results['f1_input']

In [31]:
for k in results.keys():
    if not k == 'hashcode':
        print(f'{k}',f"\n\tmean (diff): {np.mean(df_results[k+'_diff'])}\n\tstd (diff): {np.std(df_results[k+'_diff'])}\n")

precision 
	mean (diff): -0.147619934948357
	std (diff): 0.16816722939475584

recall 
	mean (diff): -0.1812424785024053
	std (diff): 0.2084570808405189

f1 
	mean (diff): -0.16638319252489564
	std (diff): 0.18992517879407717

